# 6章：機械学習の適用による予兆検知（Amazon SageMaker）

ここでは推論用のデータのデータ加工を実施します。

## ライブラリのインストールと読み込み

必要なpythonライブラリのインストールとimportを行います。

In [1]:
!pip install smart-open
!pip install tqdm

import pandas as pd
import numpy as np
import datetime as dt
import csv
import smart_open
from tqdm.notebook import tqdm
import boto3

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## データ加工

NASA Bearing Datasetのデータ加工を行います。今回のデータは推論ようです。データ加工内容は、時刻情報の追加です。\
`{your bucket name}`にご自身のS3バケット名を入力し、実行します。\
データ量が多いため、実行には数分要します。

In [2]:
bucket_name = "{your bucket name}"
rawdata_prefix_key = "IMS/3rd_test"
csvdata_prefix_key = "csv/3rd_test"

#S3から生データのファイルリストを取得
s3_client = boto3.resource('s3')
bucket = s3_client.Bucket(bucket_name)

contents = []
continuation_token = None
while True:
    if continuation_token == None:
        response = bucket.meta.client.list_objects_v2(Bucket=bucket.name, Prefix=rawdata_prefix_key)
        #print(response)
    else:
        response = bucket.meta.client.list_objects_v2(Bucket=bucket.name, Prefix=rawdata_prefix_key,ContinuationToken=continuation_token)
        #print(response)
        
    if "Contents" in response:
        contents.extend([content for content in response["Contents"]])
        continuation_token = response.get('NextContinuationToken')
        if continuation_token is not None:
            print("continue")
        else:
            print("stop")
            break
    else:
        break
            
# 各ファイルでのデータ処理
for obj in tqdm(contents):
    if obj["Size"] != 0 :
        prefixKey = obj["Key"]
        
        #ファイル名から開始時刻の取得
        filename_format = '%Y.%m.%d.%H.%M.%S'
        target_time = dt.datetime.strptime(prefixKey.split('/')[-1], filename_format)
        
        #出力するcsvファイル名、urlを定義
        csv_filename = prefixKey.split("/")[-1]+'.csv'
        url = "s3://{}/{}/{}".format(bucket_name,csvdata_prefix_key,csv_filename)
        
        #データ処理
        #S3から生データを読み込み、日付を先頭に追加してcsvファイルとしてS3に配置する。
        with smart_open.open(url, 'w', transport_params={'client': boto3.client('s3')}) as fout:
            writer = csv.writer(fout)
            with smart_open.open("s3://{}/{}".format(bucket_name,prefixKey),'rb', transport_params={'client': boto3.client('s3')}) as fin:
                for _line in fin:
                    #ファイル中のレコード読み込み
                    _line = _line.decode().replace('\r\n','').split('\t')

                    #時刻追加
                    _line.insert(0,target_time.strftime('%Y-%m-%d %H:%M:%S.%f'))

                    #レコードの追加
                    writer.writerow(_line)

                    #時間の更新
                    target_time = target_time + dt.timedelta(milliseconds=600000/20480)

stop


  0%|          | 0/106 [00:00<?, ?it/s]

### EOF